In [1]:
import numpy as np

In [3]:
l=np.load('data/processed/final_negated_entities_pizza.npy')

print(l)

['SIZE' 'NOT_STYLE' 'STYLE' 'NOT_NUMBER' 'QUANTITY' 'NOT_TOPPING' 'NUMBER'
 'TOPPING' 'NOT_QUANTITY' 'NOT_SIZE']


In [2]:
text=np.load('processed_sentences_no_conct.npy', allow_pickle=True)
entities=np.load('entities_to_words_not_processed.npy', allow_pickle=True)

In [5]:
text[0].tolist().split(' ')

['can', 'i', 'have', 'a', 'large', 'bbq', 'pulled', 'pork']

In [6]:
np.save('text_2d.npy',np.array([t.tolist().split(' ') for t in text], dtype=object))

In [9]:
t=np.load('text_2d.npy', allow_pickle=True)

In [10]:
max([len(i) for i in t])

25

In [3]:
entities=np.load('negated_entities.npy', allow_pickle=True)
entities_id = {e.item(): i+1 for i, e in enumerate(entities)}
entities_id['0']=0
entities_id['O']=0

print(entities_id)

{'TOPPING': 1, 'VOLUME': 2, 'NOT_TOPPING': 3, 'DRINKTYPE': 4, 'NOT_STYLE': 5, 'NUMBER': 6, 'QUANTITY': 7, 'CONTAINERTYPE': 8, 'SIZE': 9, 'STYLE': 10, '0': 0, 'O': 0}


In [4]:
l=np.load("tokens_to_entities.npy", allow_pickle=True)
len(l)


2456446

In [5]:
print(l[0])

[('can', 'O'), ('i', 'O'), ('have', 'O'), ('a', 'NUMBER'), ('large', 'SIZE'), ('bb', 'TOPPING'), ('##q', 'TOPPING'), ('pulled', 'TOPPING'), ('pork', 'TOPPING')]


In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/media/mohamed/AC6030326030059C/CMP1Materials/Forth/First/NLP/Project/dataset/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
tokenizer.convert_tokens_to_ids(l[0][0][0])

2064

In [6]:
input_ids_train=np.load('data/processed/train/input_ids_pizza_train.npy', allow_pickle=True)
padded_labels_drink_train=np.load('data/processed/train/padded_labels_drink_train.npy', allow_pickle=True)
padded_labels_pizza_train=np.load('data/processed/train/padded_labels_pizza_train.npy', allow_pickle=True)
input_ids_dev=np.load('data/processed/dev/input_ids_drink_dev.npy', allow_pickle=True)
padded_labels_drink_dev=np.load('data/processed/dev/padded_labels_drink_dev.npy', allow_pickle=True)
padded_labels_pizza_dev=np.load('data/processed/dev/padded_labels_pizza_dev.npy', allow_pickle=True)

# Check for NaNs
assert not np.isnan(input_ids_train).any(), "Input contains NaNs!"
assert not np.isnan(padded_labels_pizza_train).any(), "Pizza labels contain NaNs!"
assert not np.isnan(padded_labels_drink_train).any(), "Drink labels contain NaNs!"

# Check input and output ranges
assert np.max(input_ids_train) < 30522, "Input IDs exceed vocabulary size!"
assert np.min(input_ids_train) >= 0, "Input IDs contain negative values!"
assert np.max(padded_labels_pizza_train) < 21, "Pizza labels exceed class size!"
assert np.max(padded_labels_drink_train) < 21, "Drink labels exceed class size!"

In [1]:
import pandas as pd

df = pd.read_csv('test_set.csv')

df.head()

,id,order
0,0,let me prefer a extra large sauteed spinach an...
1,1,could you give me a exta large pizza with roas...
2,2,can you handle this order a pizza make it etra...
3,3,let me try 5 lnch pizzas with garlic onions an...
4,4,i need you to order me a pie in med size along...


In [3]:
num_long_orders = df['order'].apply(lambda x: len(x.split()) > 30).sum()
print(num_long_orders)

df['order'].describe()

36


count                                                  4000
unique                                                 4000
top       get me one large onion and pineapple pie with ...
freq                                                      1
Name: order, dtype: object

In [4]:
36/4000

0.009

In [ ]:
import json
import torch

# Label maps for the models
MODEL_1_LABEL_MAP = {
    "B-PIZZAORDER": 1,
    "I-PIZZAORDER": 2,
    "B-DRINKORDER": 3,
    "I-DRINKORDER": 4,
    "O": 5
}

MODEL_2_LABEL_MAP = {
    'B-DRINKTYPE': 1, 'I-DRINKTYPE': 2,
    'B-SIZE': 3, 'I-SIZE': 4,
    'B-NUMBER': 5, 'I-NUMBER': 6,
    'B-CONTAINERTYPE': 7, 'I-CONTAINERTYPE': 8,
    'B-COMPLEX_TOPPING': 9, 'I-COMPLEX_TOPPING': 10,
    'B-TOPPING': 11, 'I-TOPPING': 12,
    'B-NEG_TOPPING': 13, 'I-NEG_TOPPING': 14,
    'B-NEG_STYLE': 15, 'I-NEG_STYLE': 16,
    'B-STYLE': 17, 'I-STYLE': 18,
    'B-QUANTITY': 19, 'I-QUANTITY': 20,
    'O': 21
}

# Function to apply both models and get the TOP_DECOUPLED format


def process_entry(entry):
    src_text = entry["train.SRC"]
    true_top = entry["train.TOP"]

    # Tokenize and preprocess the input text
    tokens = tokenize_input(src_text)
    # Convert tokens to integers
    tokens = tokens_to_ints(tokens, vocab)
    # Convert to tensor
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    # Get predictions from the first model
    model_1_output = model_1(tokens)
    first_model_labels = model_1_output.argmax(
        dim=-1).squeeze(0).tolist()  # Ensure list of labels

    # Get predictions from the second model
    model_2_output = model_2(tokens)
    second_model_labels = model_2_output.argmax(
        dim=-1).squeeze(0).tolist()  # Ensure list of labels

    # Generate TOP_DECOUPLED output
    top_decoupled = generate_top_decoupled(
        src_text, first_model_labels, second_model_labels)

    # Preprocess TOP to JSON format
    predicted_json = preprocess_top(top_decoupled)
    true_json = preprocess_true_top(true_top)

    with open("predicted_true.json", "a") as f:
        f.write(f"tokens: {tokens}\n")
        f.write(f"true: {true_top}\n")
    write_comparison_file(predicted_json, true_json, top_decoupled)

    # Compare the predicted JSON with the ground truth JSON
    return predicted_json == true_json


def generate_top_decoupled(text, first_labels, second_labels):
    words = text.split()
    first_labels = first_labels[:len(words)]
    second_labels = second_labels[:len(words)]

    # Debugging output
    with open("predicted_true.json", "a") as f:
        f.write(str(words) + "\n")
        f.write(str([next(k for k, v in MODEL_1_LABEL_MAP.items() if v == l)
                for l in first_labels]) + "\n")
        f.write(str([next(k for k, v in MODEL_2_LABEL_MAP.items() if v == l)
                for l in second_labels]) + "\n\n")

    result = ["(ORDER"]
    current_order_type = None
    current_group = None
    open_groups = []  # To keep track of open groups for proper closing

    for i, (word, first_label, second_label) in enumerate(zip(words, first_labels, second_labels)):
        first_label_key = next(
            (key for key, value in MODEL_1_LABEL_MAP.items()
             if value == first_label), None
        )
        # Handle the first labels (ORDER type: PIZZAORDER, DRINKORDER)
        if first_label in [MODEL_1_LABEL_MAP["B-PIZZAORDER"], MODEL_1_LABEL_MAP["B-DRINKORDER"]]:
            if current_order_type is not None:
                result.append(")")  # Close the previous order
                open_groups.pop()  # Remove from open_groups stack
            current_order_type = "PIZZAORDER" if first_label == MODEL_1_LABEL_MAP[
                "B-PIZZAORDER"] else "DRINKORDER"
            result.append(f"({current_order_type}")
            open_groups.append(current_order_type)

        # if the first label is I- and the current order type is None, consider it as B- and add the order type
        # and do the same if it's an I- but for a different order type
        elif first_label_key.startswith("I-") and (current_order_type is None or current_order_type != first_label_key[2:]):
            if current_order_type is not None:
                result.append(")")  # Close the previous order
                open_groups.pop()
            current_order_type = "PIZZAORDER" if first_label == MODEL_1_LABEL_MAP[
                "I-PIZZAORDER"] else "DRINKORDER"
            result.append(f"({current_order_type}")
            open_groups.append(current_order_type)

        elif first_label == MODEL_1_LABEL_MAP["O"] and current_order_type is not None:
            result.append(")")  # Close the current order
            open_groups.pop()
            current_order_type = None

        elif first_label == MODEL_1_LABEL_MAP["O"] and current_order_type is None:
            continue  # Skip the word if it's not part of an order

        # Handle the second labels (attributes like NUMBER, SIZE, TOPPING, etc.)
        if second_label != MODEL_2_LABEL_MAP["O"]:
            second_label_key = next(
                (key for key, value in MODEL_2_LABEL_MAP.items()
                 if value == second_label), None
            )
            if not second_label_key:
                print(
                    f"Warning: Unexpected label {second_label} encountered for word '{word}'. Skipping.")
                continue

            label_type = second_label_key.split("-")[-1]
            if label_type not in ["NEG_TOPPING", "NEG_STYLE"]:
                if second_label_key.startswith("B-"):
                    # Close the previous group if there is one
                    if current_group:
                        result.append(")")  # Close the previous group
                        # since this is positive, if the current top group is a not group close it as well
                        if current_group["type"] == "NEG_TOPPING" or current_group["type"] == "NEG_STYLE":
                            result.append(")")
                            open_groups.pop()
                        open_groups.pop()
                    current_group = {"type": label_type, "content": [word]}
                    result.append(f"({label_type} {word}")
                    open_groups.append(label_type)

                elif second_label_key.startswith("I-") and current_group and current_group["type"] == label_type:
                    current_group["content"].append(word)
                    result[-1] += f" {word}"  # Append to the last open group

                elif second_label_key.startswith("I-") and (not current_group or current_group["type"] != label_type):
                    print(
                        f"Warning: I- tag '{label_type}' for word '{word}' without preceding B- tag. Converting to B-.")
                    # Close the previous group if there is one
                    if current_group:
                        result.append(")")  # Close the previous group
                        # since this is positive, if the current top group is a not group close it as well
                        if current_group["type"] == "NEG_TOPPING" or current_group["type"] == "NEG_STYLE":
                            result.append(")")
                            open_groups.pop()
                        open_groups.pop()
                    current_group = {"type": label_type, "content": [word]}
                    result.append(f"({label_type} {word}")
                    open_groups.append(label_type)

            # Special handling for NEG_TOPPING and NEG_STYLE
            else:
                if second_label_key.startswith("B-"):
                    if current_group:
                        result.append(")")
                        open_groups.pop()
                    result.append(
                        f"(NOT ({'TOPPING' if label_type == 'NEG_TOPPING' else 'STYLE'} {word}")
                    current_group = {"type": label_type, "content": [word]}
                    open_groups.append(label_type)
                    open_groups.append("NOT")
                elif second_label_key.startswith("I-") and current_group and current_group["type"] == label_type:
                    current_group["content"].append(word)
                    result[-1] += f" {word}"  # Append to the last open group
                elif second_label_key.startswith("I-") and (not current_group or current_group["type"] != label_type):
                    # Close the previous group if there is one
                    if current_group:
                        result.append(")")
                        open_groups.pop()
                    print(
                        f"Warning: I- tag '{label_type}' for word '{word}' without preceding B- tag. Converting to B-.")
                    result.append(
                        f"(NOT ({'TOPPING' if label_type == 'NEG_TOPPING' else 'STYLE'} {word}")
                    current_group = {"type": label_type, "content": [word]}
                    open_groups.append("NOT")
                    open_groups.append(label_type)
        # Handle O labels
        else:
            if current_group:
                result.append(")")  # Close the current group
                open_groups.pop()
            current_group = None

    # Close any remaining open groups
    while open_groups:
        result.append(")")
        open_groups.pop()

    result.append(")")  # Close the overall ORDER group
    return " ".join(result)